# Visualization(结果可视化)

**feat-1**：
> 多模态样本可视化
- ✅读取数据目录和病例目录
- ✅ 分别读取多模态数据 ['flair', 't1', 't2', 't1ce'. 'seg']
- ✅ 对中间切片进行可视化

**feat-2**
> 预处理结果可视化
- [ ] 随机裁剪可视化
- [ ] 随机旋转可视化
- [ ] 前景标准化可视化

**feat-3**：
> 多模型可视化
- ✅ 读取模型输出结果目录
- ✅获取模型原始模态数据和切片
- ✅ 获取模型预测结果的数据和切片
- ✅ 封装成对象

## 一、多模态样本可视化

### 1. 病例目录和模态数据地址

In [ ]:
import os
import random
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
COLOR_MAP = ListedColormap([(0,0,0), (1,0,0), (0,1,0), (0,0,1)], name='custom_discrete', N=4)
BOUNDARIES = [0, 1, 2, 3, 4]
NORM = BoundaryNorm(BOUNDARIES, COLOR_MAP.N)

In [ ]:

# 获取所有的病例id
def get_patient_ids(data_root):
    ids = []
    for _, dirs, _ in os.walk(data_root):
        for patient in dirs:
            ids.append(patient.split('_')[-1])
    return ids

In [ ]:
def get_all_modal_path(data_root, patient_id, modals_list):
    paths = {}
    for modal in modals_list:
        path = os.path.join(os.path.join(data_root, f"BraTS2021_{patient_id}"), f"BraTS2021_{patient_id}_{modal}.nii.gz")
        paths[modal] = path
    return paths

# paths = get_all_modal_path(data_root, patient_id, modals_list)
# print(paths)


def get_patients_paths(data_root, modals_list, ids_list):
    paths = {}
    for id in ids_list:
        path = get_all_modal_path(data_root, id, modals_list)
        paths[id]=path
    return paths





### 2.读取数据

In [ ]:
import nibabel as nib
import copy
def load_nii_data(nii_path):
    return nib.load(nii_path).get_fdata().transpose((2, 0, 1))

def load_all_data(path_dict):
    data_dict = copy.deepcopy(path_dict)
    for id, paths in path_dict.items():
        for modal, path in paths.items():
            data = load_nii_data(path)
            data_dict[id][modal] = data
    return data_dict

def get_axis_slice(data, axis, index):
    slices = [slice(None)] * data.ndim # [slice(None, None, None), slice(None, None, None), slice(None, None, None)]
    slices[axis] = index               # [slice(None, None, None), index, slice(None, None, None)]
    return data[tuple(slices)]


def get_all_slice(data_dict, slice_num, axis):
    slice_dict = copy.deepcopy(data_dict)
    for id, datas in data_dict.items():
        for modal, data in datas.items():
            slice = get_axis_slice(data, axis, slice_num)
            slice_dict[id][modal] = slice
    return slice_dict



### 3. 可视化切片

In [ ]:


def plot_data(slice_dict, axs_size=3, cmap='bone'):
    for id, id_slices in slice_dict.items():
        figsize= (axs_size*len(id_slices), axs_size)
        fig, axs = plt.subplots(1, len(id_slices), figsize=figsize)
        fig.text(0.1, 0.5, f'ID: {id}', va='center', ha='right', rotation=90, fontsize=8)
        for i, (modal, slice) in enumerate(id_slices.items()):
            if modal == 'seg':
                modal = 'GroudTruth'
                axs[i].imshow(slice, cmap=COLOR_MAP, norm=NORM, alpha=1)
                # axs[i].set_title(f'{modal}')
                axs[i].axis('off')
            else:
                axs[i].imshow(slice, cmap=cmap)
                # axs[i].set_title(f'{modal}')
                axs[i].axis('off')
        plt.show()

In [ ]:
data_root = '/root/workspace/BraTS_Solution/data/brats21/BraTS2021_Training_Data'
patient_id = '00003'

patient_path = os.path.join(data_root, f"BraTS21_{patient_id}")


modals_list = ['flair', 't1', 't2', 't1ce', 'seg']

paths = {}
# 获取所有病例的id
all_ids_list = get_patient_ids(data_root)

# 随机获取k个病例
ids_list = random.choices(all_ids_list, k=4)

ids_dict = get_patients_paths(data_root, modals_list, ids_list)

# print(ids_dict)
data_dict = load_all_data(ids_dict)
slice_dict = get_all_slice(data_dict, 96, 0)
plot_data(slice_dict)
print(data_dict.keys())


print(slice_dict)

## 二、预处理结果可视化

### 1.定义预处理方法

In [ ]:
import torch
import numpy as np
import scipy.ndimage
from scipy.ndimage import rotate


class RandomCrop3D(object):
    def __init__(self, size=(128, 128, 128)):
        self.size = size
        
    def __call__(self, image, mask):
        img = np.array(mask)
        x_start = np.random.randint(0, img.shape[-3] - self.size[0]) if img.shape[-3] > self.size[0] else 0
        y_start = np.random.randint(0, img.shape[-2] - self.size[1]) if img.shape[-2] > self.size[1] else 0
        z_start = np.random.randint(0, img.shape[-1] - self.size[2]) if img.shape[-1] > self.size[2] else 0
        
        image = image[:,x_start: x_start + self.size[0], y_start: y_start + self.size[1], z_start: z_start + self.size[2]]
        mask = mask[x_start: x_start + self.size[0], y_start: y_start + self.size[1], z_start: z_start + self.size[2]]
        return image, mask  
    
class FrontGroundNormalize(object):
    def __init__(self):
        pass
    
    def __call__(self, image, mask):
        mask = np.sum(image, axis=0) > 0 
        for k in range(4):
            x = image[k, ...]
            y = x[mask]
            if y.size > 0:
                x[mask] = (x[mask] - y.mean()) / (y.std() + 1e-6)
            image[k, ...] = x
        return image, mask


class CustomRandomRotation3D(object):
    def __init__(self, degrees=(-10, 10), axes=(0, 1, 2), p=0.5):
        """
        初始化 CustomRandomRotation3D 类
        
        参数:
        - degrees: 旋转角度范围，可以是一个元组 (min_angle, max_angle) 或者一个浮点数
        - axes: 要进行旋转的轴，可以是 (0, 1, 2) 中的任意组合
        - p: 每个轴应用旋转的概率
        """
        self.degrees = degrees if isinstance(degrees, tuple) else (-degrees, degrees)
        self.axes = axes
        self.p = p

    def __call__(self, image, mask):
        """
        对输入的图像和掩码进行随机旋转
        
        参数:
        - image: 形状为 (C, D, H, W) 的 4D numpy array
        - mask: 形状为 (D, H, W) 的 3D numpy array
        
        返回:
        - 旋转后的图像和掩码
        """
        rotated_image = image.copy()
        rotated_mask = mask.copy()

        for axis in self.axes:
            if np.random.rand() < self.p:
                angle = np.random.uniform(self.degrees[0], self.degrees[1])
                rotated_image = self.rotate(rotated_image, angle, axis)
                rotated_mask = self.rotate(rotated_mask, angle, axis, is_mask=True)

        return rotated_image, rotated_mask

    def rotate(self, x, angle, axis, is_mask=False):
        """
        沿指定轴旋转数组
        """
        axes = [(1, 2), (0, 2), (0, 1)][axis]
        if is_mask:
            return rotate(x, angle, axes=axes, reshape=False, order=0, mode='nearest')
        else:
            return np.stack([rotate(channel, angle, axes=axes, reshape=False, order=1, mode='constant', cval=0) 
                             for channel in x])
class CustomRandomFlip3D:
    def __init__(self, p=0.5, axes=("LR", "AP")):
        self.p = p
        self.axes_mapping = {"LR": -1, "AP": -2}
        self.axes_indices = []
        
        for axis in axes:
            if isinstance(axis, str):
                if axis not in self.axes_mapping:
                    raise ValueError(f"Invalid axis name: {axis}. Use 'LR', 'AP', 'IS' or integers.")
                self.axes_indices.append(self.axes_mapping[axis])
            elif isinstance(axis, int):
                self.axes_indices.append(axis)
            else:
                raise ValueError(f"Invalid axis type: {type(axis)}. Use string or integer.")

    def __call__(self, image, mask):
        flip_image = image.copy()
        flip_mask = mask.copy()
        
        # 确保至少翻转一次
        flipped = False
        for axis in self.axes_indices:
            if np.random.rand() < self.p:
                flip_image = np.flip(flip_image, axis=axis)
                flip_mask = np.flip(flip_mask, axis=axis if axis < 0 else axis-1)
                flipped = True
        
        # 如果没有发生翻转，随机选择一个轴进行翻转
        # if not flipped and self.axes_indices:
        #     axis = np.random.choice(self.axes_indices)
        #     flip_image = np.flip(flip_image, axis=axis)
        #     flip_mask = np.flip(flip_mask, axis=axis if axis < 0 else axis-1)
        
        return flip_image, flip_mask

    





#### MRI强渡归一化

In [ ]:
import numpy as np
import scipy.ndimage

class ScaleIntensityRanged(object):
    """MRI强度值归一化到制定范围(通道独立处理)"""
    def __init__(self, a_min=-100.0, 
                 a_max=3000.0, b_min=0.0, b_max=1.0
                 ):
        """
        Args:
            a_min(float):原数据截断下限
            a_max(float):原始数据截断上限
            b_min(float):归一化下限
            b_max(float):归一化上限
        """
        self.a_min = a_min
        self.a_max = a_max
        self.b_min = b_min
        self.b_max = b_max
        
    def __call__(self, image, mask):
        """
        Args:
            vimage(np.ndarray):输入图像
            vmask(np.ndarray):输入mask
        """
        
        for c in range(image.shape[0]):
            channel = image[c]
            # 1.截断异常值
            channel = np.clip(channel, self.a_min, self.a_max)
            # 2. 线性映射到目标范围
            if self.a_max != self.a_min:
                channel = (channel - self.a_min) /(self.a_max - self.a_min)
                channel = channel * (self.b_max - self.b_min) + self.b_min
            image[c] = channel
            
        return image, mask
    
    

#### 3D随机旋转增强

In [ ]:
class RandRotated(object):
    def __init__(self, prob=0.5, max_degree=10):
        self.prob = prob
        self.max_degree = max_degree
        
    def __call__(self, image, mask):
        if np.random.rand() > self.prob:
            return image, mask
        
        rot_angle = np.random.uniform(-self.max_degree, self.max_degree)
        
        def rotate_volume(vol , is_mask=False):
            rotated = np.zeros_like(vol)
            
            for c in range(vol.shape[0] if len(vol.shape)==4 else 1):
                # 随机选择旋转平面(轴向、矢状、冠状)
                plane = np.random.choice([(1, 2), (0, 2), (0, 1)])
                if is_mask:
                    rotated_slice = scipy.ndimage.rotate(
                        vol if len(vol.shape)==3 else vol[c], #?
                        angle=rot_angle,
                        axes=plane,
                        reshape=False,
                        order=0,
                        mode='constant'
                    )
                else:
                    rotated_slice = scipy.ndimage.rotate(
                        vol[c],
                        angle=rot_angle,
                        axes=plane,
                        reshape=False,
                        order=3,        # 图像用三次样条插值
                        mode='constant'
                    )
                
                # 保持尺寸不变的中心裁剪
                orig_shape = vol[c].shape if len(vol.shape)==4 else vol.shape
                crop = tuple([slice((rs-os)//2, (rs-os)//2+os) 
                            for rs, os in zip(rotated_slice.shape, orig_shape)])
                rotated_slice = rotated_slice[crop]
                if len(vol.shape)==4:
                    rotated[c] = rotated_slice
                else:
                    rotated = rotated_slice
            return rotated
        return rotate_volume(image), rotate_volume(mask, is_mask=True)

In [ ]:
class RandFlipd(object):
    """3D随机翻转增强"""
    def __init__(self, prob=0.5):
        """
        Args:
            prob (float): 沿任意轴翻转的概率
        """
        self.prob = prob

    def __call__(self, image, mask):
        # 对每个空间维度独立判断是否翻转
        for dim in range(3):
            if np.random.rand() < self.prob:
                image = np.flip(image, axis=dim+1)  # 图像维度为(C,H,W,D)
                mask = np.flip(mask, axis=dim)
        return image, mask


In [ ]:
random_crop = RandomCrop3D()
front_ground_norm = FrontGroundNormalize()
# random_noise = CustomRandomNoise3D()
random_rotation = CustomRandomRotation3D(degrees=(-30, 30), axes=(0, 1, 2), p=0.5)
random_flip = CustomRandomFlip3D()

scaleIntensityRanged = ScaleIntensityRanged()
randRotated = RandRotated()
randFlipd = RandFlipd()

### 2.处理数据

In [ ]:
def get_preprocessed_data_dict(data_dict, process_name, process_object):
    preprocessed_data_dict = {}
    
    for id, id_data in data_dict.items():
        preprocessed_data_dict[id] = {}
        data_list = []
        for modal, data in id_data.items():
            if modal == 'seg':
                vmask = data
                print(vmask)
            else:
                data_list.append(data)
                print(data)
        vimage = np.stack(data_list, axis=0)
        print(vimage)
        preprocessed_data_dict[id]['original_vimage'] = vimage
        preprocessed_data_dict[id][process_name], _ = process_object(vimage, vmask)
        
    return preprocessed_data_dict

def get_axis_slice(data, axis, index):
    # print(data.shape)
    slices = [slice(None)] * len(data.shape)
    if len(data.shape) == 4:
        slices[axis] = index    
    slice_index = data[tuple(slices)]
    print(slice_index.shape)            # [slice(None, None, None), slice(None, None, None), slice(None, None, None)]             # [slice(None, None, None), index, slice(None, None, None)]
    return slice_index

def get_all_slice(d_dict, slice_num, axis):
    slice_dict = copy.deepcopy(data_dict)
    for id, datas in d_dict.items():
        for modal, data in datas.items():
            print(modal, data.shape)
            slice_index = get_axis_slice(data, axis, slice_num)
            slice_dict[id][modal] = slice_index
    return slice_dict


scaleIntensityRangedpre_data_dict = get_preprocessed_data_dict(data_dict, 'scaleIntensityRanged', scaleIntensityRanged)
scaleIntensityRangedpre_slice_dict = get_all_slice(scaleIntensityRangedpre_data_dict, 77, 1)


# print(scaleIntensityRangedpre_data_dict)
# print(preprocessed_slice_dict)


### 3.可视化处理之后的数据

In [ ]:
def plot_slice(origin_img, preprocessed_img):
    """
    可视化一个切片
    :param img: 一个切片
    :return: None
    """
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(origin_img, cmap='gray')
    axs[0].axis('off')
    axs[1].imshow(preprocessed_img, cmap='gray')
    axs[1].axis('off')
    plt.show()
    
origin = scaleIntensityRangedpre_slice_dict[ids_list[0]]['original_vimage']
print(origin.shape)
preprocessed_data = scaleIntensityRangedpre_slice_dict[ids_list[0]]['scaleIntensityRanged']

origin == preprocessed_data


#TODO: 可视化需要继续开发
# np.unique(origin[0,...] == preprocessed_data[0,...])

## 三、多模型结果可视化

### 1. 读取模型输出结果目录

In [ ]:
def trans_from_wins_to_linux(path):
        path = path.replace('D:', '/mnt/d')
        path = path.replace('\\', '/')
        return path

In [ ]:

base_pred_cfg  = {
    "UNet3D":   trans_from_wins_to_linux(r'D:\results\compared_nets\UNet3D\UNet3D_20250308'),
    "AttnUNet": trans_from_wins_to_linux(r"D:\results\compared_nets\AttentionUNet3D\AttentionUNet3D_20250308"),
    "UNETR":    trans_from_wins_to_linux(r"D:\results\compared_nets\UNTER\UNETR_20250313222017"),
    "UNETR_PP": trans_from_wins_to_linux(r"D:\results\compared_nets\UNTER_PP\UNETR_PP_20250313224034"),
    # TODO: 添加更多网络
    "Mamba3d": trans_from_wins_to_linux(r"D:\results\compared_nets\Mamba3d\Mamba3d_20250314172525")
}

data_dir = 'D:\data\BraTS21_original_kaggle\BraTS2021_Training_Data'
ids= ['01641', '01390', '01150', '00410']

### 2.封装成对象

In [ ]:
import os
import random
import numpy as np
import pandas as pd 
import nibabel as nib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
# %matplotlib inline

COLOR_MAP = ListedColormap([(0,0,0), (1,0,0), (0,1,0), (0,0,1)], name='custom_discrete', N=4)
BOUNDARIES = [0, 1, 2, 3, 4]
NORM = BoundaryNorm(BOUNDARIES, COLOR_MAP.N)
MODALITIES = ['flair', 't1', 't2', 't1ce', 'seg']

DATA_DIR = "/root/workspace/BraTS_Solution/data/brats21/BraTS2021_Training_Data"

class MultiModalityLoader():
    def __init__(self, data_dir, modalities, transform=None):
        self.data_dir = data_dir
        self.modals = modalities
        
    def get_patient_ids(self):
        return [d.split('_')[-1] for d in os.listdir(self.data_dir) if d.startswith('BraTS2021')]
        
    def get_id_paths(self, patient_id):
        paths_dict = {}
        paths_dict[patient_id] = {}
        for modal in self.modals:
            path = os.path.join(self.data_dir, f"BraTS2021_{patient_id}", f"BraTS2021_{patient_id}_{modal}.nii.gz")
            paths_dict[patient_id][modal] = path
        return paths_dict
    
    def get_all_paths(self, length=100):
        all_paths_dict = {}
        for patient_id in self.get_patient_ids()[:length] if length else self.get_patient_ids():
            all_paths_dict[patient_id] = self.get_id_paths(patient_id)
        return all_paths_dict
    

    def __getitem__(self, patient_id):
        data = {}
        data[patient_id] = {}
        for modality in self.modals:
            path = os.path.join(self.data_dir, f"BraTS2021_{patient_id}", f"BraTS2021_{patient_id}_{modality}.nii.gz")
            if modality == 'seg':
                data[patient_id][modality] = nib.load(path).get_fdata().transpose((2, 0, 1)).astype(np.int8)
            else:
                data[patient_id][modality] = nib.load(path).get_fdata().transpose((2, 0, 1)).astype(np.float32)
        return data
    
    def __len__(self):
        return len(self.get_patient_ids())
    
    def __iter__(self):
        for patient_id in self.get_patient_ids():
            yield self[patient_id]
    
    
def plot_data(slice_dict, axs_size=3, cmap='bone'):
    for id, id_slices in slice_dict.items():
        figsize= (axs_size*len(id_slices), axs_size)
        fig, axs = plt.subplots(1, len(id_slices), figsize=figsize)
        fig.text(0.1, 0.5, f'ID: {id}', va='center', ha='right', rotation=90, fontsize=8)
        for i, (modal, slice) in enumerate(id_slices.items()):
            if modal == 'seg':
                modal = 'GroudTruth'
                axs[i].imshow(slice, cmap=COLOR_MAP, norm=NORM, alpha=1)
                # axs[i].set_title(f'{modal}')
                axs[i].axis('off')
            else:
                axs[i].imshow(slice, cmap=cmap)
                # axs[i].set_title(f'{modal}')
                axs[i].axis('off')
        plt.show()
        
class MultiModalityPloter:
    def __init__(self, data_dict, cmap='bone'):
        self.data_dict = data_dict
        self.cmap = cmap
        
    def get_slices(self, axis, slice_idx=100):
        """获取指定轴和切片的图像数据"""
        
        slice_dict = {}
        for patient_id, id_data in self.data_dict.items():
            slice_dict[patient_id] = {}
            for modal, data in id_data.items():
                slice_dict[patient_id][modal] = self.get_slice_from_axis(data, axis, slice_idx)
        return slice_dict

    def get_slice_from_axis(self, data, axis, index):
    # print(data.shape)
        slices = [slice(None)] * len(data.shape)
        slices[axis] = index    
        slice_index = data[tuple(slices)]
        print(slice_index.shape)            # [slice(None, None, None), slice(None, None, None), slice(None, None, None)]             # [slice(None, None, None), index, slice(None, None, None)]
        return slice_index
    


if __name__ == "__main__":

    loader = MultiModalityLoader(DATA_DIR, MODALITIES)



    ploter = MultiModalityPloter(loader['00018'])

    # 
    slice_dict = ploter.get_slices(0, 100)



    plot_data(slice_dict, axs_size=0, cmap='bone')


# all_paths = loader.get_all_paths()
# all_intensity = loader.get_all_IntensityRange()
# print(loader['00003']['flair'].max())
# print(loader['00003']['t1'].max())
# print(loader['00003']['t2'].max())
# print(loader['00003']['t1ce'].max())
# all_paths
# all_intensity

# plt.imshow(loader['00018']['00018']['t1'][100])

# np.unique(slice_dict['00018']['seg'])

(240, 240)
(240, 240)
(240, 240)
(240, 240)
(240, 240)


/root/miniconda3/envs/cv/lib/python3.10/site-packages/matplotlib/axes/_base.py:1999: RuntimeWarning: invalid value encountered in scalar divide
  fig_aspect = bb.height / bb.width
/root/miniconda3/envs/cv/lib/python3.10/site-packages/matplotlib/_tight_bbox.py:67: RuntimeWarning: divide by zero encountered in scalar divide
  fig.patch.set_bounds(x0 / w1, y0 / h1,
/root/miniconda3/envs/cv/lib/python3.10/site-packages/matplotlib/_tight_bbox.py:68: RuntimeWarning: divide by zero encountered in scalar divide
  fig.bbox.width / w1, fig.bbox.height / h1)
/root/miniconda3/envs/cv/lib/python3.10/site-packages/matplotlib/patches.py:796: RuntimeWarning: invalid value encountered in scalar add
  x1 = self.convert_xunits(self._x0 + self._width)
/root/miniconda3/envs/cv/lib/python3.10/site-packages/matplotlib/patches.py:797: RuntimeWarning: invalid value encountered in scalar add
  y1 = self.convert_yunits(self._y0 + self._height)


### 4. 多id可视化

In [ ]:
# class MultiMedicalVisualizer:
#     def __init__(self, original_data, pred_data):
#         """
#         初始化可视化器
#         :param original_data: 包含原始图像和GT的字典 {patient_id: {'flair':..., 'gt':...}}
#         :param pred_data: 包含预测结果的字典 {patient_id: {model_name: 3D_array...}}
#         """
#         self.original_data = original_data
#         self.pred_data = pred_data
#         self.current_patient = next(iter(original_data.keys()))
#         self.fig = None
#         tracemalloc.start()  # 启动内存监控

#     def _get_slices(self, axis, slice_idx):
#         """获取指定轴和切片的图像数据"""
#         data = {
#             'nii': self.original_data[self.current_patient]['flair'],
#             'gt': self.original_data[self.current_patient]['gt'],
#             'preds': self.pred_data[self.current_patient]
#         }
        
#         slicer = {
#             0: lambda x: x[slice_idx],
#             1: lambda x: x[:, slice_idx],
#             2: lambda x: x[:, :, slice_idx]
#         }
        
#         return {k: slicer[axis](v) if k != 'preds' else 
#                 {m: slicer[axis](d) for m, d in v.items()} 
#                 for k, v in data.items()}

#     def _plot_slice(self, axis, slice_idx):
#         """绘制单个切片"""
#         data = self._get_slices(axis, slice_idx)
        
#         # 动态计算布局
#         n_models = len(data['preds'])
#         figsize = (4*(n_models+1), 4) if axis == 0 else (4, 4*(n_models+1))
        
#         if self.fig is None:
#             self.fig, axs = plt.subplots(1, n_models+1, figsize=figsize, 
#                                        layout='compressed')
#         else:
#             self.fig.clear()
#             axs = self.fig.subplots(1, n_models+1)
        
#         # 绘制原始图像和GT
#         axs[0].imshow(data['nii'], cmap='gray')
#         gt_mask = np.ma.masked_where(data['gt'] == 0, data['gt'])
#         axs[0].imshow(gt_mask, cmap=COLOR_MAP, norm=NORM, alpha=1)
#         axs[0].set_title(f'Ground Truth\nSlice {slice_idx}')
        
#         # 绘制各模型预测
#         for i, (model_name, pred) in enumerate(data['preds'].items(), 1):
#             axs[i].imshow(data['nii'], cmap='gray')
#             pred_mask = np.ma.masked_where(pred == 0, pred)
#             axs[i].imshow(pred_mask, cmap=COLOR_MAP, norm=NORM, alpha=1)
#             axs[i].set_title(model_name)
            
#         for ax in axs:
#             ax.axis('off')
            
#         # 显示内存使用情况
#         current, peak = tracemalloc.get_traced_memory()
#         self.fig.suptitle(f"Memory usage: {current/1e6:.1f}MB (peak: {peak/1e6:.1f}MB)",
#                         y=0.95, fontsize=9, color='gray')
        
#         clear_output(wait=True)  # 清除之前的输出
#         display(self.fig)        # 异步显示

#     def interactive_view(self):
#         """生成交互式控件"""
        
#         for current_patient in self.original_data.keys():
#             self.current_patient = current_patient
#             sample_data = self.original_data[self.current_patient]['flair']
#             max_slices = {
#                 0: sample_data.shape[0]-1,
#                 1: sample_data.shape[1]-1,
#                 2: sample_data.shape[2]-1
#             }
            
#             @interact
#             def slice_explorer(
#                 Axis=['Axial (0)', 'Sagittal (1)', 'Coronal (2)'],
#                 Slice=IntSlider(min=0, max=max_slices[0], value=max_slices[0]//2)
#             ):
#                 axis = int(Axis.split('(')[1][0])
#                 self._plot_slice(axis, Slice)
            
#     def show_volume(self, axis=0, step=5):
#         """快速浏览整个体积"""
#         max_slice = self.original_data[self.current_patient]['flair'].shape[axis]-1
#         for s in range(0, max_slice+1, step):
#             self._plot_slice(axis, s)
#             plt.pause(0.1)  # 添加短暂延迟保证显示
            
#     def close(self):
#         """释放资源"""
#         if self.fig is not None:
#             plt.close(self.fig)
#         tracemalloc.stop()

# # 使用示例
# vis = MultiMedicalVisualizer(original_data_dict, pred_data_dict)
# vis.interactive_view()  # 显示交互控件
# # vis.interactive_view()
# # vis.show_volume()
# # 快速浏览模式（在单独的cell运行）
# # %matplotlib notebook
# # vis.show_volume(axis=0)


### 5. 单张可视化

In [ ]:

# import matplotlib.pyplot as plt
# from matplotlib.colors import ListedColormap, BoundaryNorm
# import numpy as np

# # 创建离散Colormap（含归一化）
# color_list = [(0,0,0), (1,0,0), (0,1,0), (0,0,1)]
# cmap = ListedColormap(color_list, name='custom_discrete', N=4)
# bounds = [0, 1, 2, 3, 4]  # 类别边界
# norm = BoundaryNorm(bounds, cmap.N)

# def get_slices(nii_data, gt_data, pred_dict, slice_idx=80, axis=0):
#     pred_slices = {}
#     if axis == 0:
#         nii_slice = nii_data[slice_idx]
#         gt_slice = gt_data[slice_idx]
#         for model_name, data in pred_dict.items():
#             pred_slices[model_name] = data[slice_idx]
#     elif axis == 1:
#         nii_slice = nii_data[:, slice_idx]
#         gt_slice = gt_data[:, slice_idx]
#         for model_name, data in pred_dict.items():
#             pred_slices[model_name] = data[:, slice_idx]
#     elif axis == 2:
#         nii_slice = nii_data[:, :, slice_idx]
#         gt_slice = gt_data[:, :, slice_idx]
#         for model_name, data in pred_dict.items():
#             pred_slices[model_name] = data[:, :, slice_idx]
        
#     return nii_slice, gt_slice, pred_slices

# def plot_slice(nii_slice, gt_slice, pred_slices, title: str, overlay_cmap: str = cmap, alpha: float = 1, axis=0):
    
#     if axis == 0:
#         figsize = (18, 10)
#     elif axis == 1 or axis == 2:
#         figsize = (10, 18)
        
#     fig, axes = plt.subplots(1, len(pred_slices)+1, figsize=figsize)
#     # 显示原始图像和GT
#     axes[0].imshow(nii_slice, cmap='gray')  # 底层图像用灰度
#     axes[0].imshow(np.ma.masked_where(gt_slice == 0, gt_slice), 
#                    cmap=overlay_cmap, norm=norm, alpha=alpha)  # GT用自定义Colormap
#     axes[0].set_title('Ground Truth')
#     axes[0].axis('off')  # 关闭坐标轴
#     # 显示预测结果
#     for i, (model_name, overlay) in enumerate(pred_slices.items()):
#         axes[i+1].imshow(nii_slice, cmap='gray')        # 底层图像用灰度
#         axes[i+1].imshow(np.ma.masked_where(overlay == 0, overlay),  # 预测结果用自定义Colormap
#                          cmap=overlay_cmap, norm=norm, alpha=alpha)
#         axes[i+1].set_title(f"{model_name}") 
#         axes[i+1].axis('off')  # 关闭坐标轴
#     fig.tight_layout()
#     axes[i+1].axis('off')  # 关闭坐标轴
#     fig.show() 


# def plot_volume(nii_data, gt_data, pred_dict, title: str, overlay_cmap: str = cmap, alpha: float = 1, axis=0):

#     for slice in range(nii_data.shape[axis]):
#         nii_slice, gt_slice, pred_slices = get_slices(nii_data, gt_data, pred_dict, slice_idx=slice, axis=axis)
#         if nii_slice.sum() != 0 and gt_slice.sum() != 0:
#             plot_slice(nii_slice, gt_slice, pred_slices, title=f'Slice {slice} {title}', overlay_cmap=overlay_cmap, alpha=alpha, axis=axis)
        

# slice_idx_axis0 = 50
# slice_idx_axis1 = 170
# slice_idx_axis2 = 50

# for patient_id in pred_data_dict.keys():
#     nii_data = original_data_dict[patient_id]['flair']
#     print(nii_data.shape)
    
#     gt_data = original_data_dict[patient_id]['gt']
#     print(gt_data.shape)
    
#     pred_dict = pred_data_dict[patient_id]
#     nii_slice, gt_slice, pred_slices = get_slices(nii_data, gt_data, pred_dict, slice_idx=slice_idx_axis0, axis=0)
#     print(nii_slice.shape, gt_slice.shape, pred_slices['UNet3D'].shape)
#     # plot_slice(nii_slice, gt_slice, pred_slices, title=f'Slice {slice_idx_axis0} Axis 横断面', axis=0)
#     # plot_volume(nii_data, gt_data, pred_dict, title='Volume')

# # for patient_id in pred_data_dict.keys():
# #     nii_data = original_data_dict[patient_id]['flair']
# #     print(nii_data.shape)
    
# #     gt_data = original_data_dict[patient_id]['gt']
# #     print(gt_data.shape)
    
# #     pred_dict = pred_data_dict[patient_id]
# #     nii_slice, gt_slice, pred_slices = get_slices(nii_data, gt_data, pred_dict, slice_idx=slice_idx_axis1, axis=1)
# #     print(nii_slice.shape, gt_slice.shape, pred_slices['UNet3D'].shape)
# #     # plot_slice(nii_slice, gt_slice, pred_slices, title=f'Slice {slice_idx_axis1} Axis 矢状面', axis=1)
# #     plot_volume(nii_data, gt_data, pred_dict, title='Volume')

# # for patient_id in pred_data_dict.keys():
# #     nii_data = original_data_dict[patient_id]['flair']
# #     print(nii_data.shape)
    
# #     gt_data = original_data_dict[patient_id]['gt']
# #     print(gt_data.shape)
    
# #     pred_dict = pred_data_dict[patient_id]
# #     nii_slice, gt_slice, pred_slices = get_slices(nii_data, gt_data, pred_dict, slice_idx=slice_idx_axis2, axis=2)
# #     print(nii_slice.shape, gt_slice.shape, pred_slices['UNet3D'].shape)
# #     # plot_slice(nii_slice, gt_slice, pred_slices, title=f'Slice {slice_idx_axis1} Axis 冠状面', axis=2)
# #     plot_volume(nii_data, gt_data, pred_dict, title='Volume')
# # # nii_slice, gt_slice, pred_slices = get_slices(nii_data, gt_data, pred_dict, slice_idx=slice_idx_axis2, axis=2)
# # # plot_slice(nii_slice, gt_slice, pred_slices, title=f'Slice {slice_idx_axis2} Axis 冠状面', axis=1)
